In [1]:
import os
import tensorflow as tf

In [2]:
# 定義train與test資料集路徑
train_path = "./train"
test_path = "./test"

In [3]:
# 定義keras內建之ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode = 'nearest',
    validation_split=0.2
)
test_generator = ImageDataGenerator(rescale=1./255)

In [4]:
# 取得train/val/test資料
train_data = train_generator.flow_from_directory(
    train_path, subset='training', target_size=(32, 32), batch_size=64, class_mode='categorical', shuffle=True
)
val_data = train_generator.flow_from_directory(
    train_path, subset='validation', target_size=(32, 32), batch_size=64, class_mode='categorical', shuffle=True
)
test_data = test_generator.flow_from_directory(
    test_path, target_size=(32, 32), batch_size=64, class_mode='categorical', shuffle=True
)

Found 40000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [5]:
# 建立CNN模型
from tensorflow.keras import datasets, layers, models
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5425047901553398850
]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
________

In [6]:
# 定義f1算法
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# 初始化優化器並設定績效計算與callback等方法
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True,
    verbose=0,
)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(train_data, epochs=200, validation_data=val_data, callbacks=[early_stop], verbose=1)

In [ ]:
# 測試模型績效
loss, accuracy, f1_score, precision, recall = model.evaluate(test_data)

In [ ]:
# model.save('/content/drive/MyDrive/Colab Notebooks/CIFAR10/model_02')

In [ ]:
 ##### 下一個模型測試，測試將relu換為tanh是否提升績效 #####

In [ ]:
# 建立CNN模型
from tensorflow.keras import datasets, layers, models
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='tanh'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
# 定義f1算法
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# 初始化優化器並設定績效計算與callback等方法
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True,
    verbose=0,
)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(train_data, epochs=200, validation_data=val_data, callbacks=[early_stop], verbose=1)

In [ ]:
# 測試模型績效
loss, accuracy, f1_score, precision, recall = model.evaluate(test_data)

In [ ]:
# model.save('/content/drive/MyDrive/Colab Notebooks/CIFAR10/model_01')

In [ ]:
 ##### 下一個模型測試，測試將relu換為selu是否提升績效 #####

In [ ]:
# 建立CNN模型
from tensorflow.keras import datasets, layers, models
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='selu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='selu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='selu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
# 定義f1算法
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# 初始化優化器並設定績效計算與callback等方法
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True,
    verbose=0,
)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(train_data, epochs=200, validation_data=val_data, callbacks=[early_stop], verbose=1)

In [ ]:
# 測試模型績效
loss, accuracy, f1_score, precision, recall = model.evaluate(test_data)

In [ ]:
# model.save('/content/drive/MyDrive/Colab Notebooks/CIFAR10/model_03')